In [1]:
# Клітинка 1: Завантаження та розархівація даних
print("Завантаження корпусу юридичних текстів... Це може зайняти кілька хвилин.")
# Використовуємо wget для завантаження файлу
!wget -q --show-progress https://static.case.law/ark/14.zip -O legal_corpus.zip

print("\nРозархівація файлу... Це також може зайняти час.")
# Розархівуємо завантажений файл
!unzip -q legal_corpus.zip -d legal_corpus

print("\nАрхів успішно розархівовано до папки 'legal_corpus'.")

Завантаження корпусу юридичних текстів... Це може зайняти кілька хвилин.
legal_corpus.zip    100%[===================>]   1.34M  --.-KB/s    in 0.06s   

Розархівація файлу... Це також може зайняти час.

Архів успішно розархівовано до папки 'legal_corpus'.


In [8]:
import os
import json

# Шлях до папки з вашими JSON файлами
json_folder_path = 'legal_corpus/json'

# Знаходимо перший-ліпший .json файл у папці
try:
    first_file_name = [f for f in os.listdir(json_folder_path) if f.endswith('.json')][0]
    file_to_inspect = os.path.join(json_folder_path, first_file_name)

    print(f"--- Аналізуємо структуру файлу: {file_to_inspect} ---")

    with open(file_to_inspect, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # Виводимо структуру JSON у читабельному вигляді
        print(json.dumps(data, indent=4))

except (FileNotFoundError, IndexError):
    print(f"Не вдалося знайти жодного .json файлу в папці {json_folder_path}")

--- Аналізуємо структуру файлу: legal_corpus/json/0421-01.json ---
{
    "id": 1862637,
    "name": "Brown vs. Collins, ad.",
    "name_abbreviation": "Brown v. Collins",
    "decision_date": "1854-01",
    "docket_number": "",
    "first_page": "421",
    "last_page": "422",
    "citations": [
        {
            "type": "official",
            "cite": "14 Ark. 421"
        }
    ],
    "court": {
        "name_abbreviation": "Ark.",
        "id": 8808,
        "name": "Arkansas Supreme Court"
    },
    "jurisdiction": {
        "id": 34,
        "name_long": "Arkansas",
        "name": "Ark."
    },
    "cites_to": [],
    "analysis": {
        "cardinality": 177,
        "char_count": 2286,
        "ocr_confidence": 0.548,
        "pagerank": {
            "raw": 1.1619320283896213e-07,
            "percentile": 0.5842392741816221
        },
        "sha256": "62c6942070cadfc032abe6a2ecc17c8d7d226ea04562d4254ea0e026357c4767",
        "simhash": "1:82de0a6e5e16b2f8",
        "word

In [9]:
# Клітинка 2 (Виправлена з урахуванням правильної структури JSON)
import os
import json
import re
from tqdm import tqdm

# Правильний шлях до папки, який ви підтвердили
corpus_path = 'legal_corpus/json'
output_corpus_file = 'corpus.txt'

file_count = 0
print(f"Починаємо обробку JSON файлів з '{corpus_path}'...")

with open(output_corpus_file, 'w', encoding='utf-8') as outfile:
    for root, dirs, files in os.walk(corpus_path):
        json_files = [f for f in files if f.endswith('.json')]
        for filename in tqdm(json_files, desc=f'Обробка папки {os.path.basename(root)}'):
            filepath = os.path.join(root, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                    # --- ВИПРАВЛЕНА ЛОГІКА ВИДОБУВАННЯ ТЕКСТУ ---
                    # Ми прибрали зайвий ключ 'data' і тепер шлях є правильним
                    case_text = data.get('casebody', {}).get('opinions', [{}])[0].get('text', '')

                    if case_text:
                        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', case_text).lower()
                        outfile.write(cleaned_text + '\n')
                        file_count += 1
            except Exception as e:
                # Ігноруємо файли з будь-якими помилками
                # print(f"Помилка при обробці файлу {filepath}: {e}") # Розкоментуйте для детальної діагностики
                continue

print(f"\nОбробку завершено. Знайдено та оброблено {file_count} файлів.")
print(f"Весь текст збережено у файл '{output_corpus_file}'.")

Починаємо обробку JSON файлів з 'legal_corpus/json'...


Обробка папки json: 100%|██████████| 129/129 [00:00<00:00, 2538.60it/s]


Обробку завершено. Знайдено та оброблено 127 файлів.
Весь текст збережено у файл 'corpus.txt'.


In [10]:
# Клітинка 3: Обробка даних та створення словників
from collections import Counter
import numpy as np
import pandas as pd

def process_data(file_name):
    """
    Зчитує файл та повертає список слів у нижньому регістрі.
    """
    with open(file_name, "r", encoding='utf-8') as f:
        # Текст вже очищений та в нижньому регістрі, тому просто читаємо слова
        text = f.read()
        words = re.findall(r'\w+', text)
    return words

def get_count(word_l):
    """
    Створює словник частотності слів.
    """
    # Використовуємо Counter для ефективного підрахунку
    word_count_dict = Counter(word_l)
    return word_count_dict

def get_probs(word_count_dict):
    """
    Обчислює ймовірності слів на основі словника частотності[cite: 13].
    """
    probs = {}
    total_count = sum(word_count_dict.values())
    for word, count in word_count_dict.items():
        probs[word] = count / total_count
    return probs

# Запускаємо процеси
print("Обробка корпусу для створення словника...")
word_l = process_data(output_corpus_file)
word_count_dict = get_count(word_l)
probs = get_probs(word_count_dict)
# Створюємо множину унікальних слів для швидкого пошуку
vocab = set(word_count_dict.keys())
print("Словники успішно створено.")
print(f"Розмір словника (кількість унікальних слів): {len(vocab)}")
print(f"Приклад ймовірності для слова 'court': {probs.get('court', 'Слово відсутнє')}")

Обробка корпусу для створення словника...
Словники успішно створено.
Розмір словника (кількість унікальних слів): 8695
Приклад ймовірності для слова 'court': 0.007085204889136512


In [12]:
# Клітинка 4: Функції для операцій редагування
def delete_letter(word):
    """
    Видаляє одну літеру зі слова та повертає список усіх можливих варіантів[cite: 14].
    """
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    return [L + R[1:] for L, R in split_l if R]

def switch_letter(word):
    """
    Міняє місцями дві сусідні літери та повертає всі можливі варіанти[cite: 15].
    """
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    return [L + R[1] + R[0] + R[2:] for L, R in split_l if len(R) > 1]

def replace_letter(word):
    """
    Замінює одну літеру іншою та повертає всі можливі варіанти[cite: 16].
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    return [L + c + R[1:] for L, R in split_l if R for c in letters]

def insert_letter(word):
    """
    Вставляє одну літеру та повертає всі можливі варіанти[cite: 17].
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    return [L + c + R for L, R in split_l for c in letters]

def edit_one_letter(word, allow_switches=True):
    """
    Повертає набір слів, які знаходяться на відстані одного редагування від вхідного слова[cite: 18].
    """
    edit_set = set(delete_letter(word))
    edit_set.update(replace_letter(word))
    edit_set.update(insert_letter(word))
    if allow_switches:
        edit_set.update(switch_letter(word))
    return edit_set

def edit_two_letters(word, allow_switches=True):
    """
    Повертає набір слів, які знаходяться на відстані двох редагувань від вхідного слова[cite: 19].
    """
    edit_set = set()
    for w in edit_one_letter(word, allow_switches):
        edit_set.update(edit_one_letter(w, allow_switches))
    return edit_set

print("Функції редагування гото-ві до використання.")
# Тестовий виклик
print(f"Приклад редагувань для слова 'lawer': {edit_one_letter('lawer')}")

Функції редагування гото-ві до використання.
Приклад редагувань для слова 'lawer': {'laweq', 'lawey', 'loawer', 'lauer', 'nawer', 'lawerc', 'lawxer', 'laweri', 'latwer', 'rlawer', 'lawuer', 'iawer', 'lawenr', 'lawel', 'lawerr', 'hlawer', 'lawver', 'wlawer', 'glawer', 'lawwer', 'laxwer', 'lawaer', 'lawehr', 'ldwer', 'lawesr', 'lager', 'mlawer', 'lfawer', 'lader', 'lawser', 'lawejr', 'lawfer', 'laper', 'lcwer', 'lawej', 'wawer', 'laweg', 'lawnr', 'cawer', 'laweru', 'laxer', 'olawer', 'lajwer', 'lauwer', 'lawekr', 'aawer', 'lawur', 'luawer', 'lnwer', 'dlawer', 'ljawer', 'lkawer', 'ltawer', 'lawyr', 'lawter', 'lawefr', 'laber', 'lhawer', 'jlawer', 'fawer', 'lawer', 'lhwer', 'laer', 'lawxr', 'lgwer', 'slawer', 'lawevr', 'lswer', 'mawer', 'lafwer', 'lawezr', 'lawerh', 'lamer', 'lawdr', 'lawea', 'laweor', 'lwer', 'qlawer', 'lqawer', 'lwaer', 'laweyr', 'lawlr', 'lbawer', 'lawzer', 'lwwer', 'laswer', 'labwer', 'lafer', 'lawemr', 'lzwer', 'laier', 'lcawer', 'lawee', 'lawcr', 'lakwer', 'lawero', 

In [13]:
# Клітинка 5: Основна функція автокорекції
def get_corrections(word, probs, vocab, n=3):
    """
    Повертає n найбільш ймовірних корекцій для введеного слова[cite: 20].
    """
    # Якщо слово вже правильне, повертаємо його з ймовірністю 1.0
    if word in vocab:
        return [(word, probs.get(word, 0))]

    # Кандидати на відстані 1
    suggestions = edit_one_letter(word)
    known_suggestions = suggestions.intersection(vocab)

    # Якщо є відомі кандидати на відстані 1, використовуємо їх
    if known_suggestions:
        # Сортуємо кандидатів за їхньою ймовірністю в корпусі
        best_words = sorted([(s, probs[s]) for s in known_suggestions], key=lambda x: x[1], reverse=True)
        return best_words[:n]

    # Кандидати на відстані 2
    suggestions_two = edit_two_letters(word)
    known_suggestions_two = suggestions_two.intersection(vocab)
    if known_suggestions_two:
        best_words = sorted([(s, probs[s]) for s in known_suggestions_two], key=lambda x: x[1], reverse=True)
        return best_words[:n]

    # Якщо нічого не знайдено, повертаємо оригінальне слово
    return [(word, 0.0)]

print("Функція автокорекції готова.")
# Тестування на прикладі
test_word = 'jurisdicion'
corrections = get_corrections(test_word, probs, vocab)
print(f"\nПропозиції для слова '{test_word}':")
for i, (correction, prob) in enumerate(corrections):
    print(f" {i+1}. '{correction}' (ймовірність: {prob:.8f})")

Функція автокорекції готова.

Пропозиції для слова 'jurisdicion':
 1. 'jurisdiction' (ймовірність: 0.00076917)


In [14]:
# Клітинка 6: Функція мінімальної відстані редагування
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    """
    Обчислює мінімальну відстань редагування між двома рядками[cite: 21].
    """
    m = len(source)
    n = len(target)
    D = np.zeros((m + 1, n + 1), dtype=int)

    # Ініціалізація першого рядка та стовпця
    for row in range(m + 1):
        D[row, 0] = row * del_cost
    for col in range(n + 1):
        D[0, col] = col * ins_cost

    # Заповнення матриці за алгоритмом динамічного програмування [cite: 11]
    for row in range(1, m + 1):
        for col in range(1, n + 1):
            r_cost = rep_cost if source[row - 1] != target[col - 1] else 0
            D[row, col] = min(
                D[row - 1, col] + del_cost,
                D[row, col - 1] + ins_cost,
                D[row - 1, col - 1] + r_cost
            )

    return D, D[m, n]

print("Функція мінімальної відстані редагування готова.")

# Тестування
source, target = 'appeal', 'apply'
distance_matrix, med = min_edit_distance(source, target)
print(f"\nМінімальна відстань редагування між '{source}' та '{target}': {med}")
print("Матриця відстаней:")
print(pd.DataFrame(distance_matrix, index=['#'] + list(source), columns=['#'] + list(target)))

Функція мінімальної відстані редагування готова.

Мінімальна відстань редагування між 'appeal' та 'apply': 3
Матриця відстаней:
   #  a  p  p  l  y
#  0  1  2  3  4  5
a  1  0  1  2  3  4
p  2  1  0  1  2  3
p  3  2  1  0  1  2
e  4  3  2  1  2  3
a  5  4  3  2  3  4
l  6  5  4  3  2  3


In [16]:
# Клітинка 7: Оцінка точності
import time

# Створюємо простий тестовий набір (слово з помилкою, правильне слово)
test_set = {
    'aproved': 'approved',
    'judgement': 'judgement',
    'constitucion': 'constitution',
    'evidance': 'evidence',
    'atorney': 'attorney',
    'defendantt': 'defendant',
    'plaintif': 'plaintiff',
    'witnes': 'witness',
    'legistation': 'legislation',
    'magistrat': 'magistrate'
}

correct_predictions = 0
total_words = len(test_set)
total_time = 0

print("--- Починаємо оцінку точності системи ---")

for wrong_word, correct_word in test_set.items():
    start_time = time.time()
    # Отримуємо найкращу пропозицію
    predictions = get_corrections(wrong_word, probs, vocab, n=1)
    end_time = time.time()

    total_time += (end_time - start_time)

    # Перевіряємо, чи є prediction і чи він правильний
    if predictions and predictions[0][0] == correct_word:
        correct_predictions += 1
        print(f"✅ Слово '{wrong_word}' виправлено правильно на '{predictions[0][0]}'")
    else:
        predicted_word = predictions[0][0] if predictions else "N/A"
        print(f"❌ Слово '{wrong_word}' виправлено НЕПРАВИЛЬНО на '{predicted_word}', очікувалося '{correct_word}'")

# Розрахунок та вивід результатів
accuracy = (correct_predictions / total_words) * 100
avg_time = total_time / total_words

print("\n--- Результати оцінки ---")
print(f"Точність системи: {accuracy:.2f}% ({correct_predictions} з {total_words} слів виправлено правильно)")
print(f"Середній час на виправлення одного слова: {avg_time:.6f} секунд")

--- Починаємо оцінку точності системи ---
❌ Слово 'aproved' виправлено НЕПРАВИЛЬНО на 'proved', очікувалося 'approved'
✅ Слово 'judgement' виправлено правильно на 'judgement'
✅ Слово 'constitucion' виправлено правильно на 'constitution'
✅ Слово 'evidance' виправлено правильно на 'evidence'
✅ Слово 'atorney' виправлено правильно на 'attorney'
✅ Слово 'defendantt' виправлено правильно на 'defendant'
✅ Слово 'plaintif' виправлено правильно на 'plaintiff'
✅ Слово 'witnes' виправлено правильно на 'witness'
✅ Слово 'legistation' виправлено правильно на 'legislation'
✅ Слово 'magistrat' виправлено правильно на 'magistrate'

--- Результати оцінки ---
Точність системи: 90.00% (9 з 10 слів виправлено правильно)
Середній час на виправлення одного слова: 0.000265 секунд
